In [124]:
# Download libraries for processing XML and HTML with Python

!pip install lxml

import lxml.html as lh

import requests

import pandas as pd

In [125]:
# Retrieve/download the file from the web-site

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Check the document on the web-site.
# All the table data ( headers, postal codes) are stored between the HTML tags

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of each row, it must have 3 columns
[len(T) for T in tr_elements]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 12,
 18]

In [126]:
print('\n')
print('# Rows loaded into a list after download :',len(tr_elements))
print('\n')



# Rows loaded into a list after download : 185




In [127]:
#Create empty list  to simply generate the header information
# tr_elements[0]  contains the column headers
# List 'col' will simply be a list of 3 column headers, and an empty list for each header.

col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
print('\n')
print('Header of list: ')
print(col)
print('\n')



Header of list: 
[('Postal Code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]




In [128]:
#Since the first row is the header, data is stored on the second row onwards
# List 'col' is now appended with values for each of the 3 columns

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
     
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
                
        #If row is not of size 3, the //tr data is not from our table of postal codes
        # This step will filter out any unwanted data such as another table on the web page
    
        if len(T)!=3:
           break
             
        if i==0:
            # If Postal Code is empty, then data is not from the table of postal codes
            if len(data.strip())==0:
                break
        
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        
        #Increment i for the next column
        i+=1

In [129]:
# Checking the final list and verifying rows in each column
print('\n')
print('Checking the final list and verifying rows in each column :')
[len(C) for (title,C) in col]



Checking the final list and verifying rows in each column :


[180, 180, 180]

In [130]:
# Converting the list into a data frame

Dict={title:column for (title,column) in col}

df=pd.DataFrame(Dict)

In [131]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [132]:
df.shape

(180, 3)

In [133]:
# Check the tail of the data frame
df.tail()

,Postal Code\n,Borough\n,Neighborhood\n
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,\n


In [134]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

# Count of rows with 'Borough' that is 'Not assigned'

print('\n')
print('Count of rows for each value of Boroughs  :\n', df['Borough\n'].value_counts())
print('\n')
df_Borough_To_drop = df.loc[df['Borough\n'] == 'Not assigned\n']
print('\n')
print('Boroughs to drop due to --Not Assigned-- values :')
print('\n')
print(df_Borough_To_drop)

# Drop rows with ''Borough'' that is ''Not assigned'', and then reset the index

df.drop(df.loc[df['Borough\n'] == 'Not assigned\n'].index, inplace=True)

df = df.reset_index(drop=True)
print('\n')
print('Size of dataframe after removing  --Borough-- that is --Not assigned--')
print('# Rows, Columns',df.shape)
print('\n')



Count of rows for each value of Boroughs  :
 Not assigned\n        77
North York\n          24
Downtown Toronto\n    19
Scarborough\n         17
Etobicoke\n           12
Central Toronto\n      9
West Toronto\n         6
East York\n            5
East Toronto\n         5
York\n                 5
Mississauga\n          1
Name: Borough\n, dtype: int64




Boroughs to drop due to --Not Assigned-- values :


    Postal Code\n       Borough\n Neighborhood\n
0           M1A\n  Not assigned\n             \n
1           M2A\n  Not assigned\n             \n
7           M8A\n  Not assigned\n             \n
10          M2B\n  Not assigned\n             \n
15          M7B\n  Not assigned\n             \n
..            ...             ...            ...
174         M4Z\n  Not assigned\n             \n
175         M5Z\n  Not assigned\n             \n
176         M6Z\n  Not assigned\n             \n
177         M7Z\n  Not assigned\n             \n
179         M9Z\n  Not assigned\n             \n

[77

In [135]:
# Remove the extraneous characters'\n' in every row, column 

df['Postal Code'] = df['Postal Code\n'].str.rstrip('\n')
df['Borough'] = df['Borough\n'].str.rstrip('\n')
df['Neighborhood'] = df['Neighborhood\n'].str.rstrip('\n')

df_new = df[['Postal Code','Borough','Neighborhood']]
df_new.head()
print('\n')
print ('Number of rows columns in the revised data frame', df_new.shape)
print('\n')



Number of rows columns in the revised data frame (103, 3)




In [136]:
# Check for rows where column Neighborhood has value has 'Not assigned'

df_Neighbor_NA = df_new.loc[df['Neighborhood'] == 'Not assigned']

#  Display the data frame that shows list of rows where 'Neighborhood' equals 'Not assigned'
print('\n')
print('Do any rows exist with no values for column Neighborhood ? ',df_Neighbor_NA)
print('\n')



Do any rows exist with no values for column Neighborhood ?  Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []




In [137]:
# Final data frame size after scrapping data

print('\n')
print('Rows and Columns of the final data frame : ', df_new.shape)
print('\n')



Rows and Columns of the final data frame :  (103, 3)




In [ ]:
# Check the data to ensure the right records are displayed

In [138]:
df_new.iloc[0:9]

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [114]:
df_new.iloc[10:19]

,Postal Code,Borough,Neighborhood
10,M6B,North York,Glencairn
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov..."
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
13,M3C,North York,Don Mills
14,M4C,East York,Woodbine Heights
15,M5C,Downtown Toronto,St. James Town
16,M6C,York,Humewood-Cedarvale
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [115]:
df_new.iloc[20:29]

,Postal Code,Borough,Neighborhood
20,M5E,Downtown Toronto,Berczy Park
21,M6E,York,Caledonia-Fairbanks
22,M1G,Scarborough,Woburn
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
25,M6G,Downtown Toronto,Christie
26,M1H,Scarborough,Cedarbrae
27,M2H,North York,Hillcrest Village
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"


In [116]:
df_new.iloc[30:39]

,Postal Code,Borough,Neighborhood
30,M5H,Downtown Toronto,"Richmond, Adelaide, King"
31,M6H,West Toronto,"Dufferin, Dovercourt Village"
32,M1J,Scarborough,Scarborough Village
33,M2J,North York,"Fairview, Henry Farm, Oriole"
34,M3J,North York,"Northwood Park, York University"
35,M4J,East York,"East Toronto, Broadview North (Old East York)"
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
37,M6J,West Toronto,"Little Portugal, Trinity"
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"


In [117]:
df_new.iloc[40:49]

,Postal Code,Borough,Neighborhood
40,M3K,North York,Downsview
41,M4K,East Toronto,"The Danforth West, Riverdale"
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
43,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
45,M2L,North York,"York Mills, Silver Hills"
46,M3L,North York,Downsview
47,M4L,East Toronto,"India Bazaar, The Beaches West"
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"


In [118]:
df_new.iloc[50:59]

,Postal Code,Borough,Neighborhood
50,M9L,North York,Humber Summit
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
52,M2M,North York,"Willowdale, Newtonbrook"
53,M3M,North York,Downsview
54,M4M,East Toronto,Studio District
55,M5M,North York,"Bedford Park, Lawrence Manor East"
56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
57,M9M,North York,"Humberlea, Emery"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [119]:
df_new.iloc[60:69]

,Postal Code,Borough,Neighborhood
60,M3N,North York,Downsview
61,M4N,Central Toronto,Lawrence Park
62,M5N,Central Toronto,Roselawn
63,M6N,York,"Runnymede, The Junction North"
64,M9N,York,Weston
65,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town..."
66,M2P,North York,York Mills West
67,M4P,Central Toronto,Davisville North
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"


In [120]:
df_new.iloc[70:79]

,Postal Code,Borough,Neighborhood
70,M9P,Etobicoke,Westmount
71,M1R,Scarborough,"Wexford, Maryvale"
72,M2R,North York,"Willowdale, Willowdale West"
73,M4R,Central Toronto,"North Toronto West, Lawrence Park"
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
75,M6R,West Toronto,"Parkdale, Roncesvalles"
76,M7R,Mississauga,Canada Post Gateway Processing Centre
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
78,M1S,Scarborough,Agincourt


In [121]:
df_new.iloc[80:89]

,Postal Code,Borough,Neighborhood
80,M5S,Downtown Toronto,"University of Toronto, Harbord"
81,M6S,West Toronto,"Runnymede, Swansea"
82,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan"
83,M4T,Central Toronto,"Moore Park, Summerhill East"
84,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
87,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"


In [122]:
df_new.iloc[90:99]

,Postal Code,Borough,Neighborhood
90,M1W,Scarborough,"Steeles West, L'Amoreaux West"
91,M4W,Downtown Toronto,Rosedale
92,M5W,Downtown Toronto,Stn A PO Boxes
93,M8W,Etobicoke,"Alderwood, Long Branch"
94,M9W,Etobicoke,"Northwest, West Humber - Clairville"
95,M1X,Scarborough,Upper Rouge
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
97,M5X,Downtown Toronto,"First Canadian Place, Underground city"
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"


In [123]:
df_new.iloc[100:105]

,Postal Code,Borough,Neighborhood
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
